In [1]:
import numpy as np
from ondelette import wavelets
import pywt
import tensorflow as tf

np.set_printoptions(floatmode='unique', suppress=True)
rng = np.random.default_rng()

2025-03-01 14:53:15.597455: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
dtype = tf.float64
wavelet_name = 'db3'

wavelet = wavelets.by_name(wavelet_name, dtype=dtype)
wavelet_width = wavelet.shape[0]

sample_width = 68 #wavelet_width
while True:
    samples_np = rng.integers(-32768, 32767, [2, sample_width], dtype=np.int16, endpoint=True)
    samples = tf.constant(samples_np, dtype=dtype)
    dec = wavelets.wavedec(wavelet, 6, samples)
    evens = [level.shape[1] % 2 == 0 for level in dec]
    if all(evens):
        break
    sample_width += 2

print(f'SAMPLES: {samples.shape}')
print(samples.numpy())
print()

for i, level in enumerate(dec):
    print(f'level {i}: {level.shape}')
    # print(level.numpy())

rec = wavelets.waverec(wavelet, dec)
print(f'RECONSTRUCTION: {rec.shape}')
print(rec.numpy())


approximation: (2, 36)
[[    81.01170242630394   1603.7283397959284   24067.764526946146
    7342.401794434658     7296.983556512355   -14243.107578993693
   -7586.951147733544    16523.208678750805   -19673.07459810974
  -21428.490296087024    -4410.42849317024    -11143.257084630342
  -16883.276706461722    24567.07606559872     -1065.5607062506392
    7450.408773487106   -14485.231842247993   -34348.440424048036
  -12832.775008999963   -16753.463489006517   -19664.92864679505
  -14269.631793347855   -20730.855779166315    -4635.344020871283
    1726.458739567539     6872.107268950875   -27454.031513748258
   13916.238544232101     6984.684015417845    44189.16033205821
  -28854.09764102357     12572.079039228223     2063.4209984738627
   -7627.679582071641     4852.575190211808     6456.13523512421   ]
 [  2650.296023543132   -15618.36737992448     25084.06539661587
    5976.017863158932    -8377.947150844408     8414.915965520771
  -23431.560449006367    42701.362031715915    -2573

In [17]:

a4, d4, d3, d2, d1, d0 = dec

r3 = wavelets.idwt(wavelet, a4, d4)
print(f'r3: {r3.shape}')
# print(r3.numpy())
pad_a = 4
pad_d = 2
r3 = tf.pad(r3, [[0, 0], [pad_a, pad_a]])
d3 = tf.pad(d3, [[0, 0], [pad_d, pad_d]])
r2 = wavelets.idwt(wavelet, r3, d3)
print(f'r2: {r2.shape}')
# print(r2.numpy())
pad_a = 2
pad_d = 4
r2 = tf.pad(r2, [[0, 0], [pad_a, pad_a]])
d2 = tf.pad(d2, [[0, 0], [pad_d, pad_d]])
r1 = wavelets.idwt(wavelet, r2, d2)
print(f'r1: {r1.shape}')
# print(r1.numpy())
pad_a = 2
pad_d = 8
r1 = tf.pad(r1, [[0, 0], [pad_a, pad_a]])
d1 = tf.pad(d1, [[0, 0], [pad_d, pad_d]])
r0 = wavelets.idwt(wavelet, r1, d1)
print(f'r0: {r0.shape}')
# print(r0.numpy())
pad_a = 2
pad_d = 16
r0 = tf.pad(r0, [[0, 0], [pad_a, pad_a]])
d0 = tf.pad(d0, [[0, 0], [pad_d, pad_d]])
rec = wavelets.idwt(wavelet, r0, d0)
print(f'RECONSTRUCTION: {rec.shape}')
print(rec.numpy())


r3: (2, 8)
r2: (2, 28)
r1: (2, 60)
r0: (2, 124)
RECONSTRUCTION: (2, 252)
[[     6.303662693687855                  15.28951649009052
      24.003580984418264                  34.526401586779635
      43.52052236625812                   52.6830132068355
      63.78162424701897                   75.62107641453484
      85.08422152511876                   94.08843720940463
     103.97739207874702                  113.77611480752441
     125.43205727987026                  137.9296421691372
     150.2607764398312                   163.11215121326794
     258.7076916378765                   389.7368127749878
     517.7355429827776                   669.8112303761286
     802.2599790598779                   937.3194163093646
    1098.124731525686                   1269.0299651820842
     678.526531472174                   -228.4950191338682
   -1092.0278784834409                 -2166.006152400366
   -4353.660283453646                  -7109.192957834036
  -11472.084978490722                